In [2]:
import os
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.callbacks import TensorBoard


def lenet(input_shape, num_classes):
    model = Sequential()

    # extract image features by convolution and max pooling layers
    model.add(Conv2D(
        20, kernel_size=5, padding="same",
        input_shape=input_shape, activation="relu"
        ))   # 5x5のフィルター20個を使用して畳み込み。出力は28x28x20
    model.add(MaxPooling2D(pool_size=(2, 2)))   # 特徴マップのサイズを圧縮。処理後の特徴マップは14x14x20
    model.add(Conv2D(50, kernel_size=5, padding="same", activation="relu"))   # 出力は14x14x50
    model.add(MaxPooling2D(pool_size=(2, 2)))   # 出力は7x7x50
    # classify the class by fully-connected layers
    model.add(Flatten())   # マトリクスのデータである特徴マップをベクトルに変換し、後続の全結合層とつなぐ。
    model.add(Dense(500, activation="relu"))   # サイズ500のベクトルを出力。
    model.add(Dense(num_classes))
    model.add(Activation("softmax"))   # 値を0~1の確率値に変換
    return model


class MNISTDataset():

    def __init__(self):
        self.image_shape = (28, 28, 1)  # image is 28x28x1 (grayscale)
        self.num_classes = 10

    def get_batch(self):
        (x_train, y_train), (x_test, y_test) = mnist.load_data()

        x_train, x_test = [self.preprocess(d) for d in [x_train, x_test]]
        y_train, y_test = [self.preprocess(d, label_data=True) for d in
                           [y_train, y_test]]

        return x_train, y_train, x_test, y_test

    def preprocess(self, data, label_data=False):
        if label_data:
            # convert class vectors to binary class matrices
            data = keras.utils.to_categorical(data, self.num_classes)
        else:
            data = data.astype("float32")
            data /= 255  # convert the value to 0~1 scale
            shape = (data.shape[0],) + self.image_shape  # add dataset length
            data = data.reshape(shape)

        return data


class Trainer():

    def __init__(self, model, loss, optimizer):
        self._target = model
        self._target.compile(
            loss=loss, optimizer=optimizer, metrics=["accuracy"]
            )
        self.verbose = 1
        logdir = "logdir_lenet"
        self.log_dir = os.path.join(os.path.dirname('__file__'), logdir)

    def train(self, x_train, y_train, batch_size, epochs, validation_split):
        if os.path.exists(self.log_dir):
            import shutil
            shutil.rmtree(self.log_dir)  # remove previous execution
        os.mkdir(self.log_dir)

        self._target.fit(
            x_train, y_train,
            batch_size=batch_size, epochs=epochs,
            validation_split=validation_split,
            callbacks=[TensorBoard(log_dir=self.log_dir)],
            verbose=self.verbose
        )


dataset = MNISTDataset()

# make model
model = lenet(dataset.image_shape, dataset.num_classes)

# train the model
x_train, y_train, x_test, y_test = dataset.get_batch()
trainer = Trainer(model, loss="categorical_crossentropy", optimizer=Adam())
trainer.train(
    x_train, y_train, batch_size=128, epochs=12, validation_split=0.2
    )

# show result
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


Train on 48000 samples, validate on 12000 samples


Epoch 1/12
48000/48000 [==============================] - 102s 2ms/step - loss: 0.1931 - accuracy: 0.9405 - val_loss: 0.0609 - val_accuracy: 0.9816

Epoch 2/12
48000/48000 [==============================] - 102s 2ms/step - loss: 0.0501 - accuracy: 0.9847 - val_loss: 0.0435 - val_accuracy: 0.9878
Epoch 3/12
48000/48000 [==============================] - 101s 2ms/step - loss: 0.0332 - accuracy: 0.9898 - val_loss: 0.0391 - val_accuracy: 0.9881
Epoch 4/12
48000/48000 [==============================] - 101s 2ms/step - loss: 0.0252 - accuracy: 0.9916 - val_loss: 0.0339 - val_accuracy: 0.9896
Epoch 5/12
48000/48000 [==============================] - 101s 2ms/step - loss: 0.0183 - accuracy: 0.9943 - val_loss: 0.0362 - val_accuracy: 0.9910
Epoch 6/12
48000/48000 [==============================] - 101s 2ms/step - loss: 0.0134 - accuracy: 0.9956 - val_loss: 0.0391 - val_accuracy: 0.9902
Epoch 7/12
48000/48000 [==============================] - 